In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
#Hasta 15 de Abril (para entrenar)

postulantes_edu_df = pd.read_csv('fiuba_1_postulantes_educacion_h15.csv')
postulantes_gye_df = pd.read_csv('fiuba_2_postulantes_genero_y_edad_h15.csv')
vistas_df = pd.read_csv('fiuba_3_vistas_h15.csv')
postulaciones_df = pd.read_csv('fiuba_4_postulaciones_h15.csv')
avisos_online_df = pd.read_csv('fiuba_5_avisos_online_h15.csv')
avisos_detalle_df = pd.read_csv('fiuba_6_avisos_detalle_h15.csv')

In [3]:
#Originales TP1 (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas.csv')
postulaciones_df2 = pd.read_csv('fiuba_4_postulaciones.csv')
avisos_online_df2 = pd.read_csv('fiuba_5_avisos_online.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle.csv')

In [4]:
#Realizo el Merge del tipo outer de los datos
#Originales TP1 y Datos hasta 15 de abril

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
postulaciones_df = postulaciones_df.merge(postulaciones_df2,how='outer')
avisos_online_df = avisos_online_df.merge(avisos_online_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')

In [5]:
#Desde 15 de abril (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion_d15.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad_d15.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas_d15.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle_d15.csv')

In [6]:
#Avisos detalle que faltaban
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [7]:
#Mergeamos todo

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_missing,how='outer')

# Libero Espacio de la Memoria

In [8]:
del postulantes_edu_df2
del postulantes_gye_df2
del vistas_df2
del avisos_detalle_df2

# Filtrado de Postulantes con informacion completa

In [9]:
postulantes_edu_df.head(2)

,idpostulante,nombre,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso


In [10]:
postulantes_gye_df.head(2)

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA


In [11]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

In [12]:
postulantes_edu_df.estado.value_counts()

Graduado      451142
En Curso      182338
Abandonado     60161
Name: estado, dtype: int64

In [13]:
postulantes_edu_df.nombre.value_counts()

Universitario        250850
Secundario           244689
Terciario/Técnico    113421
Otro                  53363
Posgrado              20624
Master                10074
Doctorado               620
Name: nombre, dtype: int64

In [14]:
print(postulantes_edu_df.shape)
print(postulantes_gye_df.shape)

(693641, 3)
(505382, 3)


In [15]:
print(postulantes_edu_df.idpostulante.unique().shape)
print(postulantes_gye_df.idpostulante.unique().shape)

(447909,)
(504407,)


In [16]:
postulantes_edu_df['estado'].value_counts()

Graduado      451142
En Curso      182338
Abandonado     60161
Name: estado, dtype: int64

# Trabajamos con la educacion

Asignamos un valor para cada estado de la educacion, Graduado->2, En Curso->1, Abandonado->0

In [17]:
def setCategory(c):
    if c == 'Graduado':
        return 2
    elif c == 'En Curso':
        return 1
    return 0 

In [18]:
postulantes_edu_df['valor_estado'] = postulantes_edu_df['estado'].apply(setCategory)

In [19]:
postulantes_edu_df.head()

,idpostulante,nombre,estado,valor_estado
0,ZjlZ,Master,En Curso,1
1,NdJl,Posgrado,En Curso,1
2,5kNq,Otro,En Curso,1
3,8rYD,Master,En Curso,1
4,1Wvj,Universitario,En Curso,1


In [20]:
postulantes_edu_df['nombre'].value_counts()

Universitario        250850
Secundario           244689
Terciario/Técnico    113421
Otro                  53363
Posgrado              20624
Master                10074
Doctorado               620
Name: nombre, dtype: int64

Al igual que hicimos con los estados de la educacion, lo hacemos con el nivel de educacion.

In [21]:
def setCategory(c):
    
    if c == 'Doctorado':
        return 12
    elif c == 'Master':
        return 10
    elif c == 'Posgrado':
        return 8
    elif c == 'Universitario':
        return 6
    elif c == 'Terciario/Técnico':
        return 4
    elif c == 'Secundario':
        return 2
    return 0 

In [22]:
postulantes_edu_df['valor_nombre'] = postulantes_edu_df['nombre'].apply(setCategory)

In [23]:
postulantes_edu_df.head(2)

,idpostulante,nombre,estado,valor_estado,valor_nombre
0,ZjlZ,Master,En Curso,1,10
1,NdJl,Posgrado,En Curso,1,8


Ahora unificamos los datos obtenidos, creando una escala donde a mayor educacion, mayor peso. Considerando el tipo de educacion y su estado.
Este nuevo feature llamado valoracion_edu, reemplazará a 'nombre'y 'estado'

In [24]:
postulantes_edu_df['valoracion_edu'] = postulantes_edu_df['valor_estado'] + postulantes_edu_df['valor_nombre']
postulantes_edu_df.drop(columns=['valor_estado','valor_nombre','nombre','estado'],axis=1,inplace=True)
postulantes_edu_df.sort_values(by='valoracion_edu',ascending=False,inplace=True)
postulantes_edu_df.drop_duplicates(inplace=True)
postulantes_edu_df.head()

,idpostulante,valoracion_edu
318724,N9AN45,14
128733,vV9Ewjj,14
337840,aMbWjD,14
398665,OqmK81N,14
14753,JBrVvBp,14


In [25]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Los valores 0.0 los unificamos con NO_DECLARA

In [26]:
def setCategory(c):
    if c == '0.0':
        return 'NO_DECLARA'
    return c

In [27]:
postulantes_gye_df['sexo'] = postulantes_gye_df['sexo'].apply(setCategory)

In [28]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25943
Name: sexo, dtype: int64

# Unificamos los dataframes GYE con EDU

In [29]:
#Para la informacion faltante agregamos una Categoria NO_DECLARA
postulantes_filtrados_df = postulantes_gye_df.merge(postulantes_edu_df,on='idpostulante',how='outer')

print(postulantes_filtrados_df.shape)

postulantes_filtrados_df['idpostulante'].unique().shape

(738053, 4)


(504407,)

In [30]:
postulantes_filtrados_df.head(2)

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,NaN
1,Nzz,NaN,NO_DECLARA,NaN


Asumimos que los usuarios que no completaron su valor, tienen educacion minima, es decir, secundario abandonado.
Las fechas de nacimiento nulas se completa con la edad promedio de los usuarios registrados(31 años).

In [31]:
postulantes_filtrados_df['sexo'].fillna('NO_DECLARA',inplace=True)
postulantes_filtrados_df['valoracion_edu'].fillna(2, inplace=True)
postulantes_filtrados_df['fechanacimiento'].fillna('1987-01-01', inplace=True)

In [32]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,2.0
1,Nzz,1987-01-01,NO_DECLARA,2.0
2,ZX1,1987-01-01,NO_DECLARA,2.0
3,Nq5,1987-01-01,NO_DECLARA,2.0
4,ebE,1952-07-07,MASC,10.0


In [33]:
postulantes_filtrados_df.shape

(738053, 4)

In [34]:
postulantes_filtrados_df.sort_values(by='sexo',inplace=True)

Al ordenar alfabeticamente sexo, los No_declara quedan ultimos, por lo tanto, al hacer un drop_duplicates, nos quedamos con la primera aparicion de ese usuario.
De esta forma si un usuario se registro dos veces, queda el registro correcto sobre su genero.

In [35]:
postulantes_filtrados_df = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes_filtrados_df.shape

(504407, 4)

In [36]:
postulantes_filtrados_df.sexo.value_counts()

FEM           251226
MASC          227832
NO_DECLARA     25349
Name: sexo, dtype: int64

In [37]:
postulantes_filtrados_df.idpostulante.unique().shape

(504407,)

In [38]:
postulantes_filtrados_df.head(2)

,idpostulante,fechanacimiento,sexo,valoracion_edu
369026,5mPYWjp,1989-05-01,FEM,4.0
588433,xkdxkDz,1993-08-28,FEM,5.0


In [39]:
postulantes_filtrados_df.fechanacimiento.describe()

count         504407
unique         17246
top       1987-01-01
freq           26120
Name: fechanacimiento, dtype: object

Las edades incorrectas son reemplazadas por al edad promedio.

In [40]:
#Le asigno la fecha promedio a las fechas incorrectas
postulantes_edad_trucha = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == True]
postulantes_edad_trucha['fechanacimiento'] = '1987-01-01'
postulantes_edad_trucha.head()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,idpostulante,fechanacimiento,sexo,valoracion_edu
299748,8M2R6pz,1987-01-01,FEM,2.0
311804,0zkAXBE,1987-01-01,FEM,7.0
341475,VNrP8EE,1987-01-01,FEM,2.0
347580,xkdB9JK,1987-01-01,FEM,8.0
324043,VNxMRxj,1987-01-01,FEM,2.0


In [41]:
#Filtro la edad incorrecta y hago un append de la edad corregida.
postulantes_filtrados_df = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == False]
postulantes_filtrados_df = postulantes_filtrados_df.append(postulantes_edad_trucha)
postulantes_filtrados_df.head()


,idpostulante,fechanacimiento,sexo,valoracion_edu
369026,5mPYWjp,1989-05-01,FEM,4.0
588433,xkdxkDz,1993-08-28,FEM,5.0
303865,EzE23E4,1999-04-23,FEM,4.0
303863,6r5OwJO,1997-06-07,FEM,4.0
588434,lDL3D1X,1997-07-17,FEM,2.0


In [42]:
#Verifico que no se hayan perdido datos.
postulantes_filtrados_df.shape

(504407, 4)

Calculo la edad y reemplazo la columna 'fechanacimiento' por 'edad'

In [43]:
postulantes_filtrados_df.loc[:,'fechanacimiento'] = pd.to_datetime(postulantes_filtrados_df['fechanacimiento']) 
postulantes_filtrados_df.loc[:,'edad'] = (2018 - postulantes_filtrados_df.fechanacimiento.dt.year)
postulantes_filtrados_df.drop(columns='fechanacimiento',axis=1,inplace=True)
postulantes_filtrados_df.head()
#Generar edad por bloques, como queremos hacer con las empresas mas populares.

,idpostulante,sexo,valoracion_edu,edad
369026,5mPYWjp,FEM,4.0,29
588433,xkdxkDz,FEM,5.0,25
303865,EzE23E4,FEM,4.0,19
303863,6r5OwJO,FEM,4.0,21
588434,lDL3D1X,FEM,2.0,21


In [44]:
postulantes_filtrados_df.head(2)

,idpostulante,sexo,valoracion_edu,edad
369026,5mPYWjp,FEM,4.0,29
588433,xkdxkDz,FEM,5.0,25


# Avisos Detalle

In [45]:
avisos_detalle_df.head(2)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan


In [46]:
print(avisos_detalle_df.shape)
print(avisos_detalle_df['idaviso'].unique().shape)

(25901, 11)
(25288,)


In [47]:
#Sacamos los repetidos de avisos_detalle
avisos_detalle_filtrados_df = avisos_detalle_df.drop_duplicates('idaviso')
avisos_detalle_df.shape

(25901, 11)

In [48]:
#No se usa mas, a partir de ahora usamos avisos detalle filtrados.
del avisos_detalle_df

In [49]:
avisos_detalle_filtrados_df.shape

(25288, 11)

#Hago un drop de las columnas que no voy a utilizar.
#Mas adelanto debo modificar esto porque la idea es usar descripcion para nuevos features.

In [50]:
avisos_detalle_filtrados_df= avisos_detalle_filtrados_df.drop(columns=['idpais','mapacalle','ciudad','descripcion'],axis=1)

In [51]:
avisos_detalle_filtrados_df.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


# Postulaciones

In [52]:
postulantes_filtrados_df['idpostulante'].unique().shape

(504407,)

In [53]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


Comparamos los "usuarios" con los  postulantes.

In [54]:
print(postulantes_filtrados_df.idpostulante.unique().shape)
print(postulaciones_df.idpostulante.unique().shape)

(504407,)
(348883,)


Como resultado tenemos los postulantes que coinciden en postulaciones y postulantes
Observamos que hay usuarios que no se postularon a ningun aviso.

In [55]:
postulantes_filtrados_efectivos_df = postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')
postulantes_filtrados_efectivos_df.idpostulante.unique().shape

(348883,)

In [56]:
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.drop_duplicates('idpostulante')
postulantes_filtrados_efectivos_df.shape

(348883, 6)

In [57]:
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop(['idaviso','fechapostulacion'],axis='columns')
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.reset_index()
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop('index',axis='columns')
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,valoracion_edu,edad
0,NjlD,FEM,6.0,37
1,ZaO5,FEM,6.0,40
2,NdJl,MASC,8.0,49
3,5kNq,MASC,1.0,45
4,Ez8J,MASC,6.0,50


Nos quedamos con los usuarios sobre los cuales tenemos informacion y ademas se postularon a un aviso.

In [58]:
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,valoracion_edu,edad
0,NjlD,FEM,6.0,37
1,ZaO5,FEM,6.0,40
2,NdJl,MASC,8.0,49
3,5kNq,MASC,1.0,45
4,Ez8J,MASC,6.0,50


In [59]:
postulantes_filtrados_efectivos_df.shape

(348883, 4)

# Armado del set de Postulaciones

Primero armo un dataframe con la informacion de postulaciones que ya tengo -> postulantes_df.
Observar que aca puede haber postulantes sobre los cuales no tengo informacion sobre genero y edad.
Tambien puede haber postulantes que luego no aparezcan en el test final.

Me quedo con los idavisos sobre los cuales tengo informacion

In [60]:
avisos = avisos_detalle_filtrados_df['idaviso'].to_frame()

Me quedo con los idpostulantes sobre los cuales tengo informacion

In [61]:
postulantes = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes = postulantes['idpostulante'].to_frame()

Hago una interseccion para quedarme con usuarios y avisos sobre los cuales tengo informacion

In [62]:
postulantes_df =  postulaciones_df.merge(avisos,how='inner',on='idaviso')
postulantes_df = postulantes_df.merge(postulantes,how='inner',on='idpostulante')

Armo el formato necesario para que me quede idpostulante-idaviso

In [63]:
postulantes_df =postulantes_df.groupby('idpostulante').agg({'idaviso':'value_counts'})
postulantes_df.columns=['postulacion']
postulantes_df['postulacion'] = 1 
avisos = 0
postulantes = 0

In [64]:
postulantes_df.head()

postulacion
idpostulante idaviso                
0z5Dmrd      1112384041            1
             1112420060            1
0z5JW1r      1112257171            1
             1112281879            1
             1112292169            1

In [65]:
postulantes_df.reset_index(inplace=True)

In [66]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [67]:
postulantes_df.idaviso.unique().shape

(18695,)

# Armado del set de No Postulaciones

Armarmos un set de no postulaciones asignando de forma random avisos a usuarios.
Mas adelante se verifica que esta no postulacion al azar no sea una postulacion efectiva.

# No agarrar postulaciones o vistas

In [68]:
vistas_df.head()

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0
2,1112426664,2018-04-06T01:43:50.042-0400,GNOwZmb
3,1112426664,2018-04-06T01:43:51.171-0400,GNOwZmb
4,1112445738,2018-04-06T01:43:46.371-0400,lDLxajz


In [69]:
vistas_df = vistas_df.drop('timestamp',axis=1)
vistas_df['lo_vio']=1
vistas_df.head()

,idAviso,idpostulante,lo_vio
0,1112400775,kPjr2K0,1
1,1112400775,kPjr2K0,1
2,1112426664,GNOwZmb,1
3,1112426664,GNOwZmb,1
4,1112445738,lDLxajz,1


In [70]:
vistas_df.columns = ['idaviso','idpostulante','lo_vio']


In [71]:
vistas_df = vistas_df.drop_duplicates(['idaviso','idpostulante'])

In [72]:
vistas_df.shape

(8597915, 3)

In [73]:
lista = []
user_list = postulantes_filtrados_df['idpostulante'].unique() #Utilizo a todos los usuarios. 
aviso_list = avisos_detalle_filtrados_df['idaviso'].unique() #Utilizo todos los avisos.
index = 1
for n in range(5000000):
    user_index = random.randint(0,504371)
    aviso_index = random.randint(0,25287)
    lista.append((index,user_list[user_index],aviso_list[aviso_index],0))
    index+=1        

In [74]:
no_postulantes_df = pd.DataFrame(lista)

In [75]:
del lista

In [76]:
no_postulantes_df.memory_usage(deep=True).sum()

438943207

In [77]:
no_postulantes_df.head()

,0,1,2,3
0,1,owajlBr,1112494268,0
1,2,JBmK2eb,1111978182,0
2,3,Ymp9bP,1112490564,0
3,4,Oqr36m6,1111202474,0
4,5,5MJXow,1112502959,0


In [78]:
no_postulantes_df.drop(0,axis='columns',inplace=True)

In [79]:
no_postulantes_df.head()

,1,2,3
0,owajlBr,1112494268,0
1,JBmK2eb,1111978182,0
2,Ymp9bP,1112490564,0
3,Oqr36m6,1111202474,0
4,5MJXow,1112502959,0


In [80]:
no_postulantes_df.columns= ['idpostulante','idaviso','postulacion']

In [81]:
no_postulantes_df.shape

(5000000, 3)

In [82]:
no_postulantes_df.head()

,idpostulante,idaviso,postulacion
0,owajlBr,1112494268,0
1,JBmK2eb,1111978182,0
2,Ymp9bP,1112490564,0
3,Oqr36m6,1111202474,0
4,5MJXow,1112502959,0


In [83]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


# Armado de Set de Entrenamiento

Quitamos las no postulaciones que figuran como vistas.

In [84]:
entrenamiento_df = vistas_df.merge(no_postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [85]:
entrenamiento_df.head()

,idaviso,idpostulante,lo_vio,postulacion
0,1112400775,kPjr2K0,1.0,NaN
1,1112426664,GNOwZmb,1.0,NaN
2,1112445738,lDLxajz,1.0,NaN
3,1112416123,owzZOqe,1.0,NaN
4,1112443938,NGk68l,1.0,NaN


In [86]:
entrenamiento_df['lo_vio'].hasnans

True

In [87]:
entrenamiento_df['postulacion'].fillna(0,inplace=True)
entrenamiento_df['lo_vio'].fillna(0,inplace=True)

In [88]:
entrenamiento_df['postulacion'] = entrenamiento_df['lo_vio'] + entrenamiento_df['postulacion']

In [89]:
entrenamiento_df.head()

,idaviso,idpostulante,lo_vio,postulacion
0,1112400775,kPjr2K0,1.0,1.0
1,1112426664,GNOwZmb,1.0,1.0
2,1112445738,lDLxajz,1.0,1.0
3,1112416123,owzZOqe,1.0,1.0
4,1112443938,NGk68l,1.0,1.0


In [90]:
entrenamiento_df = entrenamiento_df[entrenamiento_df['postulacion']==0]

In [91]:
entrenamiento_df = entrenamiento_df.drop('lo_vio',axis=1)

Juntamos los no_postulantes con los postulantes para armar el set de entrenamiento

In [92]:
entrenamiento_df = entrenamiento_df.merge(postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [93]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion_x,postulacion_y
0,1112494268,owajlBr,0.0,NaN
1,1111978182,JBmK2eb,0.0,NaN
2,1112490564,Ymp9bP,0.0,NaN
3,1111202474,Oqr36m6,0.0,NaN
4,1112502959,5MJXow,0.0,NaN


In [94]:
entrenamiento_df['postulacion_x'].fillna(0,inplace=True)
entrenamiento_df['postulacion_y'].fillna(0,inplace=True)

La idea es sumar las columnas duplicadas, ya que si habia una postulacion para esa misma dupla idaviso-idpostulante, predomina la postulacion(1) sobre la no_postulacion(0)

In [95]:
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion_x'] + entrenamiento_df['postulacion_y']

In [96]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion_x,postulacion_y,postulacion
0,1112494268,owajlBr,0.0,0.0,0.0
1,1111978182,JBmK2eb,0.0,0.0,0.0
2,1112490564,Ymp9bP,0.0,0.0,0.0
3,1111202474,Oqr36m6,0.0,0.0,0.0
4,1112502959,5MJXow,0.0,0.0,0.0


In [97]:
 entrenamiento_df.drop(['postulacion_x','postulacion_y'],axis='columns',inplace=True)

In [98]:
entrenamiento_df['postulacion'].value_counts()

1.0    6187929
0.0    4996143
Name: postulacion, dtype: int64

In [99]:
entrenamiento_df['postulacion'] = pd.Categorical(entrenamiento_df['postulacion'])
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion'].cat.codes
entrenamiento_df['postulacion'].value_counts()

1    6187929
0    4996143
Name: postulacion, dtype: int64

In [100]:
entrenamiento_df[entrenamiento_df['postulacion']==0].merge(vistas_df,on=['idaviso','idpostulante'],how='inner').shape

(0, 4)

In [101]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion
0,1112494268,owajlBr,0
1,1111978182,JBmK2eb,0
2,1112490564,Ymp9bP,0
3,1111202474,Oqr36m6,0
4,1112502959,5MJXow,0


In [102]:
entrenamiento_df.tail()

,idaviso,idpostulante,postulacion
11184067,1112438823,zvxzrWm,1
11184068,1112442213,zvxzrWm,1
11184069,1112445486,zvxzrWm,1
11184070,1112448822,zvxzrWm,1
11184071,1112455515,zvxzrWm,1


In [103]:
entrenamiento_df.idaviso.unique().shape

(25288,)

# Armo la Matriz de Features

In [52]:
del postulantes_edu_df
del postulantes_gye_df
del postulantes_edad_trucha
del avisos_detalle_missing
del postulaciones_df2

In [53]:
#Para levantar los csv guardados previamente.


entrenamiento_df=pd.read_csv('entrenamiento_df_1')
entrenamiento_df.drop('Unnamed: 0',axis=1,inplace=True)
#avisos_detalle_filtrados_df=pd.read_csv('avisos_detalle_filtrados_df')
#postulantes_filtrados_df=postulantes_filtrados_nuevo


Me quedo con un Sample de 1M

In [54]:
entrenamiento_df = entrenamiento_df.sample(100000)

In [55]:
entrenamiento_df.postulacion.value_counts()

1    55146
0    44854
Name: postulacion, dtype: int64

# En este paso vamos a crear una nueva columna en avisos_detalle donde indica el valor de la valoracion_edu promedio de cada aviso segun los postulantes.

En primer lugar, agrego la informacion de cada aviso a las postulaciones

In [56]:
merge_detalle_postulaciones_df = postulaciones_df.merge(avisos_detalle_filtrados_df, on = 'idaviso',how='inner')

In [94]:
#merge_detalle_postulaciones_df = merge_detalle_postulaciones_df.sample(1000)

In [95]:
merge_detalle_postulaciones_df.head(2)

,idaviso,idpostulante,fechapostulacion,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112248724,NjlD,2018-01-19 07:39:16,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
1,1112248724,MV6eM8N,2018-01-30 23:44:30,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack


Esto ya lo corri para todos los datos. Dejo el csv de esta matriz en el archivo en el git y todo el codigo comentado

In [107]:
# Se puede encontrar la educacion promedio para cada aviso y luego asignar esa educacion a los postulantes que no tienen educacion definida
postulantes = postulantes_filtrados_efectivos_df.groupby('idpostulante')

In [43]:
dictionary = {}
def armarFeatureConValoracionEduPorAviso(idpostulante,idaviso):
    if not idaviso in dictionary:
        dictionary[idaviso] = [0,0] #El primer elemento es la suma acumulada de valoracion de edad. El segundo es contador
    val_edu = dictionary[idaviso]
    val_edu[1]+=1
    val_edu[0] += postulantes.get_group(idpostulante)['valoracion_edu'].iloc[0]
    dictionary[idaviso] = val_edu
   

In [98]:
merge_detalle_postulaciones_df.apply(lambda row: armarFeatureConValoracionEduPorAviso(row['idpostulante'], row['idaviso']),axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
6188604    None
6188605    None
6188606    None
6188607    None
6188608    None
6188609    None
6188610    None
6188611    None
6188612    None
6188613    None
6188614    None
6188615    None
6188616    None
6188617    None
6188618    None
6188619    None
6188620    None
6188621    None
6188622    None
6188623    None
6188624    None
6188625    None
6188626    None
6188627    None
6188628    None
6188629    None
6188630    None
6188631    None
6188632    None
6188633    None
Length: 6188634, dtype: 

In [99]:
for key,value in dictionary.items():
    prom = round((value[0]/value[1]))
    dictionary[key] = prom

In [100]:
def assignValEduToAdd(c):
    return dictionary[c]

In [101]:
merge_detalle_postulaciones_df['val_edu'] = merge_detalle_postulaciones_df['idaviso'].apply(assignValEduToAdd)

In [102]:
merge_detalle_postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,val_edu
0,1112248724,NjlD,2018-01-19 07:39:16,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0
1,1112248724,MV6eM8N,2018-01-30 23:44:30,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0
2,1112248724,5LzQkN,2018-01-15 07:23:49,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0
3,1112248724,8M2rOBz,2018-01-30 22:20:45,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0
4,1112248724,Dr6V3Da,2018-01-23 15:11:35,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0


In [103]:
#merge_detalle_postulaciones_df = pd.read_csv('merge_detalle_postulaciones_df.csv')

In [104]:
#merge_detalle_postulaciones_df.drop('Unnamed: 0',1,inplace=True)

# En este paso vamos a crear una nueva columna en avisos_detalle donde indica el valor de la edad promedio de cada aviso segun los postulantes.

In [105]:
dictionary = {}
def armarFeatureConEdadPorAviso(idpostulante,idaviso):
    if not idaviso in dictionary:
        dictionary[idaviso] = [0,0] #El primer elemento es la suma acumulada de  edad. El segundo es contador
    val_edad = dictionary[idaviso]
    val_edad[1]+=1
    val_edad[0] += postulantes.get_group(idpostulante)['edad'].iloc[0]
    dictionary[idaviso] = val_edad
   

In [106]:
merge_detalle_postulaciones_df.apply(lambda row: armarFeatureConEdadPorAviso(row['idpostulante'], row['idaviso']),axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
6188604    None
6188605    None
6188606    None
6188607    None
6188608    None
6188609    None
6188610    None
6188611    None
6188612    None
6188613    None
6188614    None
6188615    None
6188616    None
6188617    None
6188618    None
6188619    None
6188620    None
6188621    None
6188622    None
6188623    None
6188624    None
6188625    None
6188626    None
6188627    None
6188628    None
6188629    None
6188630    None
6188631    None
6188632    None
6188633    None
Length: 6188634, dtype: 

In [107]:
for key,value in dictionary.items():
    prom = round((value[0]/value[1]))
    dictionary[key] = prom

In [108]:
def assignValEdadToAdd(c):
    return dictionary[c]

In [109]:
merge_detalle_postulaciones_df['val_edad'] = merge_detalle_postulaciones_df['idaviso'].apply(assignValEdadToAdd)

In [110]:
merge_detalle_postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,val_edu,val_edad
0,1112248724,NjlD,2018-01-19 07:39:16,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
1,1112248724,MV6eM8N,2018-01-30 23:44:30,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
2,1112248724,5LzQkN,2018-01-15 07:23:49,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
3,1112248724,8M2rOBz,2018-01-30 22:20:45,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
4,1112248724,Dr6V3Da,2018-01-23 15:11:35,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0


In [111]:
merge_detalle_postulaciones_df = merge_detalle_postulaciones_df.merge(avisos_detalle_filtrados_df.drop(columns=['nombre_zona','tipo_de_trabajo','nivel_laboral','denominacion_empresa','nombre_area','titulo'],axis=1),how='inner',on='idaviso')

In [112]:
merge_detalle_postulaciones_df.nombre_zona.value_counts()

Gran Buenos Aires              5632222
Capital Federal                 537203
GBA Oeste                         6972
Buenos Aires (fuera de GBA)       6039
Cordoba                           5281
Corrientes                         917
Name: nombre_zona, dtype: int64

In [113]:
merge_detalle_postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,val_edu,val_edad
0,1112248724,NjlD,2018-01-19 07:39:16,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
1,1112248724,MV6eM8N,2018-01-30 23:44:30,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
2,1112248724,5LzQkN,2018-01-15 07:23:49,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
3,1112248724,8M2rOBz,2018-01-30 22:20:45,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0
4,1112248724,Dr6V3Da,2018-01-23 15:11:35,Analista de CRM y Marketing Digital,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,6.0,30.0


In [114]:
#merge_detalle_postulaciones_df.to_csv('merge_postulaciones_detalle.csv')

In [80]:
#merge_detalle_postulaciones_df = pd.read_csv('merge_postulaciones_detalle.csv')

# Comenzamos a dar formato para armar los features

En primer lugar agrupo todos los avisos a los que se postulo un usuario

In [57]:
grupo_postulantes = merge_detalle_postulaciones_df.groupby('idpostulante')

En segundo lugar hago que 'idaviso' sea el indice para poder acceder de forma directa.

In [58]:
grupo_avisos = avisos_detalle_filtrados_df.groupby('idaviso')

Armo lista de los usuarios que se postularon a algun aviso del que yo tenga datos en avisos_detalle_filtrados_df

In [59]:
usuarios_con_postulaciones = grupo_postulantes.groups.keys()

# Feature Postulacion A Zona (Arreglar)

In [84]:
#La idea es aplicar lo mismo que para las areas, pero con las zonas.
def armarFeaturePostulacionesUsuarioAEsaZona(idpostulante,idaviso,ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    zona = grupo_avisos.get_group(idaviso)['nombre_zona'].iloc[0]
    zonas = grupo_postulantes.get_group(idpostulante)['nombre_zona'].value_counts()
    if zonas.index.contains(zona):
        return (zonas[zona]-ajuste)
    return 0

In [119]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaZona(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [120]:
entrenamiento_df['postulaciones_a_esa_zona'] = pd.DataFrame(feature)

# Feature Postulacion A Area

In [100]:
#Cuento las postulaciones por area de cada postulante sobre el cual tengo informacion
def armarFeaturePostulacionesUsuarioAEseArea(idpostulante,idaviso,ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    area = grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    areas = grupo_postulantes.get_group(idpostulante)['nombre_area'].value_counts()
    if areas.index.contains(area):
        return (areas[area]-ajuste)
    return 0

In [101]:
#Tarda mucho, por eso me quedé solo con 100000 datos del entrenamiento_df
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseArea(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [102]:
entrenamiento_df['postulaciones_a_ese_area'] = pd.DataFrame(feature)

# Variante de feature postulacion a ese area

## A ese Area En esa Zona

In [60]:
def armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(idpostulante,idaviso,ajuste):
    if (idpostulante not in usuarios_con_postulaciones):
        return 0
    area=grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    zona=grupo_avisos.get_group(idaviso)['nombre_zona'].iloc[0]
    postulaciones_de_usuario = grupo_postulantes.get_group(idpostulante).groupby('nombre_area')
    if area in postulaciones_de_usuario['nombre_area'].groups.keys():
        zonas=postulaciones_de_usuario.get_group(area)['nombre_zona'].value_counts()
        if zonas.index.contains(zona):
            return(zonas[zona]-ajuste)
    return 0

In [61]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [62]:
entrenamiento_df['postulaciones_a_ese_area_en_esa_zona'] = pd.DataFrame(feature)

# Feature Postulacion A Empresa

In [63]:
def armarFeaturePostulacionesUsuarioAEsaEmpresa(idpostulante,idaviso,ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    empresa = grupo_avisos.get_group(idaviso)['denominacion_empresa'].iloc[0]
    empresas = grupo_postulantes.get_group(idpostulante)['denominacion_empresa'].value_counts()
    if empresas.index.contains(empresa):
        return (empresas[empresa]-ajuste)
    return 0

In [64]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [65]:
entrenamiento_df['postulaciones_a_esa_empresa'] = pd.DataFrame(feature)

# Feature Educacion

In [87]:
g_avisos = merge_detalle_postulaciones_df.groupby('idaviso')

In [88]:
def armarFeatureEduDelAvisoYPostulante(idpostulante,idaviso):
    if ((idpostulante in usuarios_con_postulaciones) == False) or  not (idaviso in g_avisos.groups.keys()) :
        return 0
    val_edu = g_avisos.get_group(idaviso)['val_edu'].iloc[0]
    valoracion_edu = postulantes.get_group(idpostulante)['valoracion_edu'].iloc[0]
    if abs(val_edu-valoracion_edu) < 3:
        return 1
    return 0

In [131]:
feature = entrenamiento_df.apply(lambda row: armarFeatureEduDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [132]:
entrenamiento_df['relacion_educacion_con_aviso'] = pd.DataFrame(feature)

# Feature Edad

In [89]:
def armarFeatureEdadDelAvisoYPostulante(idpostulante,idaviso):
    if ((idpostulante in usuarios_con_postulaciones) == False) or  not (idaviso in g_avisos.groups.keys()) :
        return 0
    val_edad = g_avisos.get_group(idaviso)['val_edad'].iloc[0]
    edad = postulantes.get_group(idpostulante)['edad'].iloc[0]
    if abs(val_edad-edad) <=3:
        return 1
    return 0

In [134]:
feature = entrenamiento_df.apply(lambda row: armarFeatureEduDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [135]:
entrenamiento_df['relacion_edad_con_aviso'] = pd.DataFrame(feature)

In [85]:
import unicodedata
def shave_marks(txt):
    """This method removes all diacritic marks from the given string"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)


# Feature Titulo

In [87]:
from collections import Counter
def armarFeatureInteresEnPalabrasDeTitulo(idpostulante, idaviso, ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    palabras_en_titulos_de_sus_postulaciones = shave_marks(" ".join(grupo_postulantes.get_group(idpostulante)['titulo'].values.tolist()).lower())
    #Saco comas, parentesis, sufijos etc. Para abarcar mas palabras
    titulo_aviso = shave_marks(grupo_avisos.get_group(idaviso)['titulo'].iloc[0].lower()).replace("(",' ').replace(")",' ').replace(",",' ').replace('/',' ').replace(' para ',' ').replace('-',' ').replace(' con ',' ').replace('experiencia',' ').replace('zona',' ').replace('villa',' ').replace('barrio',' ').replace('ero ',' ').replace('era ',' ').replace('cion ',' ').replace('ativo ',' ').replace('ativa ',' ').replace('dor ',' ').replace('dora ',' ').replace('dores ',' ').replace('doras ',' ').replace('ista ',' ').replace('or ',' ').replace('ora ',' ').replace('ico ',' ').replace('ica ',' ').replace('icos ',' ').replace('icas ',' ').replace('time',' ').replace('area',' ').split(" ")
    contador=0
    for x in titulo_aviso:
        if  len(x)>2 and x!='' and palabras_en_titulos_de_sus_postulaciones.count(x) > 0:
            contador = contador + palabras_en_titulos_de_sus_postulaciones.count(x) - ajuste
    return contador

In [88]:
feature = entrenamiento_df.apply(lambda row: armarFeatureInteresEnPalabrasDeTitulo(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [89]:
entrenamiento_df['interes_en_titulo_de_aviso'] = pd.DataFrame(feature)

In [104]:
entrenamiento_df[(entrenamiento_df['postulacion']==1) & (entrenamiento_df['postulaciones_a_ese_area_en_esa_zona']==0) & (entrenamiento_df['postulaciones_a_esa_empresa']==0) & (entrenamiento_df['interes_en_titulo_de_aviso']==0)].shape

(2424, 6)

In [132]:
entrenamiento_df[(entrenamiento_df['postulacion']==1) & (entrenamiento_df['postulaciones_a_ese_area_en_esa_zona']==0) & (entrenamiento_df['postulaciones_a_esa_empresa']==0) & (entrenamiento_df['interes_en_titulo_de_aviso']==0)].sample(10)

,idpostulante,idaviso,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
10777296,vVj853E,1112407256,1,0,0,0
6777873,BmapYxR,1112280868,1,0,0,0
8567534,X95BmY8,1112365193,1,0,0,0
9368483,bO41Nv4,1112335409,1,0,0,0
7508683,LNK6Gzk,1112035266,1,0,0,0
6035860,6r5jKol,1112407034,1,0,0,0
7021071,EzExJv8,1110921809,1,0,0,0
7653037,MVajGRK,1112220908,1,0,0,0
6803604,DrJ0x4P,1112084995,1,0,0,0
6094304,6rPZq3M,1112334806,1,0,0,0


In [138]:
entrenamiento_df=entrenamiento_df[(entrenamiento_df['postulacion']!=1) | (entrenamiento_df['postulaciones_a_ese_area']!=0) | (entrenamiento_df['postulaciones_a_esa_empresa']!=0) | (entrenamiento_df['interes_en_titulo_de_aviso']!=0)]

In [139]:
matrix_df = entrenamiento_df

In [140]:
matrix_df.head()

,idpostulante,idaviso,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
5421355,2zQb8oQ,1112107934,1,2,0,2
2781224,dYj4N42,1112018104,0,0,0,0
1723283,1Qr3OBB,1112299492,0,0,0,0
4082659,xkvO9Qx,1112278940,0,0,0,0
1922810,JBxqKzW,1112222805,0,2,0,0


In [141]:
matrix_df.idaviso.shape

(97576,)

In [142]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

True

In [143]:
#Ver que está ocupando memoria.
%who

Counter	 armarFeatureInteresEnPalabrasDeTitulo	 armarFeaturePostulacionesUsuarioAEsaEmpresa	 armarFeaturePostulacionesUsuarioAEseArea	 armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona	 avisos_detalle_filtrados_df	 avisos_online_df	 avisos_online_df2	 entrenamiento_df	 
feature	 grupo_avisos	 grupo_postulantes	 matrix_df	 merge_detalle_postulaciones_df	 np	 pd	 postulaciones_df	 postulantes_filtrados_df	 
random	 setCategory	 shave_marks	 unicodedata	 usuarios_con_postulaciones	 


Me quedo con el indice como "id" de la tupla postulante-aviso.

In [144]:
matrix_df.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

/home/luciano/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
matrix_df.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
5421355,1,2,0,2
2781224,0,0,0,0
1723283,0,0,0,0
4082659,0,0,0,0
1922810,0,2,0,0


In [146]:
matrix_df['postulacion'].value_counts()

1    52722
0    44854
Name: postulacion, dtype: int64

In [147]:
X = matrix_df

In [148]:
X.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
5421355,1,2,0,2
2781224,0,0,0,0
1723283,0,0,0,0
4082659,0,0,0,0
1922810,0,2,0,0


In [122]:
#X = pd.read_csv('training_set.csv')
#X.head()

,Unnamed: 0,postulacion,postulaciones_a_esa_zona,postulaciones_a_ese_area,postulaciones_a_esa_empresa,relacion_educacion_con_aviso,relacion_edad_con_aviso,interes_en_titulo_de_aviso,idpostulante,idaviso
0,0,0,0,0,0,0,0,0,QNE15QW,1112418892
1,1,0,0,0,0,0,0,0,3Nk85BJ,1112274050
2,2,0,0,0,0,0,0,0,PmGzY61,1112227151
3,3,0,0,0,0,0,1,0,lD5OeYz,1112425167
4,4,0,2,0,0,1,1,0,zvwXrO3,1112251605


In [123]:
#X = X.drop(['Unnamed: 0','idpostulante','idaviso'],axis=1)

In [93]:
#X = X[X.postulacion == 1].head(1000000).append(X[X.postulacion == 0].head(1000000))


In [149]:
y = X['postulacion']
X.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
5421355,1,2,0,2
2781224,0,0,0,0
1723283,0,0,0,0
4082659,0,0,0,0
1922810,0,2,0,0


In [150]:
X = X.drop('postulacion',axis=1)

In [151]:
X.shape

(97576, 3)

In [152]:
y.shape

(97576,)

# Test Final DF

In [153]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [154]:
test_final_df.head(2)

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL


In [98]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaZona(row['idpostulante'], row['idaviso'],0),axis=1)

In [99]:
test_final_df['postulaciones_a_esa_zona'] = pd.DataFrame(feature)

In [155]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseArea(row['idpostulante'], row['idaviso'],0),axis=1)

In [156]:
test_final_df['postulaciones_a_ese_area']=pd.DataFrame(feature)

In [90]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(row['idpostulante'], row['idaviso'],0),axis=1)
test_final_df['postulaciones_a_ese_area_en_esa_zona']=pd.DataFrame(feature)

In [91]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEseNivelLaboral(row['idpostulante'], row['idaviso'],0),axis=1)
test_final_df['postulaciones_a_ese_area_en_ese_nivel']=pd.DataFrame(feature)

In [157]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area
0,0,739260,6M9ZQR,0
1,1,739260,6v1xdL,5


In [158]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso'],0),axis=1)

In [159]:
test_final_df['postulaciones_a_esa_empresa']=pd.DataFrame(feature)

In [160]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area,postulaciones_a_esa_empresa
0,0,739260,6M9ZQR,0,0
1,1,739260,6v1xdL,5,0


In [108]:
feature = test_final_df.apply(lambda row: armarFeatureEduDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [109]:
test_final_df['relacion_educacion_con_aviso'] = pd.DataFrame(feature)

In [110]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_esa_zona,postulaciones_a_ese_area,postulaciones_a_esa_empresa,relacion_educacion_con_aviso
0,0,739260,6M9ZQR,0,0,0,0
1,1,739260,6v1xdL,4,5,0,0


In [111]:
feature = test_final_df.apply(lambda row: armarFeatureEdadDelAvisoYPostulante(row['idpostulante'], row['idaviso']),axis=1)

In [112]:
test_final_df['relacion_edad_con_aviso'] = pd.DataFrame(feature)

In [161]:
feature = test_final_df.apply(lambda row: armarFeatureInteresEnPalabrasDeTitulo(row['idpostulante'], row['idaviso'],0),axis=1)

In [162]:
test_final_df['interes_en_titulo_de_aviso']=pd.DataFrame(feature)

In [163]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
0,0,739260,6M9ZQR,0,0,0
1,1,739260,6v1xdL,5,0,0


In [166]:
test_final_df[(test_final_df['postulaciones_a_ese_area']==0) & (test_final_df['postulaciones_a_esa_empresa']==0) & (test_final_df['interes_en_titulo_de_aviso']==0)].shape

(47285, 6)

Chequeamos que esten todos los datos.


In [187]:
test_final_df.shape

(100000, 6)

In [188]:
test_final_df = test_final_df.drop(['id','idaviso','idpostulante'],axis=1)

In [189]:
test_final_df.head()

,postulaciones_a_ese_area,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
0,0,0,0
1,5,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [119]:
#test_final_df.to_csv('test_final_df.csv')

In [142]:
#test_final_df = pd.read_csv('test_final_df.csv')
#test_final_df.drop('Unnamed: 0',1,inplace=True)

In [190]:
test_final_df.head()

,postulaciones_a_ese_area,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
0,0,0,0
1,5,0,0
2,0,0,0
3,0,0,0
4,0,0,0


# Visualizacion del set de etrenamiento

In [419]:
import matplotlib.pyplot as plt


In [541]:
#plt.scatter(X[:,0], X[:,1], alpha=0.4, c=trainLabels)
#plt.title("Datos de 'entrenamiento'")
#plt.show()

In [31]:
X.head()

,postulaciones_a_esa_zona,postulaciones_a_ese_area,postulaciones_a_esa_empresa,relacion_educacion_con_aviso,relacion_edad_con_aviso,interes_en_titulo_de_aviso
341,205,124,1,1,0,182
905,17,23,0,1,0,53
2338,14,8,0,0,1,3
6305,3,0,1,0,0,2
16281,72,1,1,1,0,9


In [33]:
test_final_df.head()

,postulaciones_a_ese_area,postulaciones_a_esa_empresa,relacion_educacion_con_aviso,interes_en_titulo_de_aviso,postulaciones_a_esa_zona
0,0,0,0,0,0
1,5,0,4,0,4
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


# XGBoost

In [120]:
from sklearn.model_selection import cross_val_score

In [121]:
import xgboost as xgb

In [128]:
xgbclassiffier = xgb.XGBClassifier(max_depth=4, learning_rate=0.1)

In [39]:
scores = cross_val_score(xgbclassiffier,X,y)
scores.mean()  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8995355009472982

In [129]:
xgbclassiffier.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [132]:
pred = xgbclassiffier.predict_proba(test_final_df)

In [133]:
pred = pred[:,1]

In [157]:
list(zip(X[X.columns], xgbclassiffier.feature_importances_))

[('postulaciones_a_esa_zona', 0.20666666),
 ('postulaciones_a_ese_area', 0.15133333),
 ('postulaciones_a_esa_empresa', 0.12733333),
 ('relacion_educacion_con_aviso', 0.154),
 ('relacion_edad_con_aviso', 0.104666665),
 ('interes_en_titulo_de_aviso', 0.256)]

# Random Forest

In [406]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

In [407]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [408]:
list(zip(X[X.columns], clf.feature_importances_))

[('postulaciones_a_ese_area', 0.4881500396704431),
 ('postulaciones_a_esa_empresa', 0.023675258915016303),
 ('relacion_educacion_con_aviso', 0.09644698650180368),
 ('interes_en_titulo_de_aviso', 0.39172771491273684)]

In [409]:
pred = clf.predict_proba(test_final_df)

pred = pred[:,1]

# Random Forest con Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=clf,n_estimators=10)

In [398]:
bc.fit(X,y)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [542]:
pred = bc.predict_proba(test_final_df)

In [ ]:
pred = pred[:,1]

# Perceptron

In [217]:
from sklearn.linear_model import Perceptron
ptron = Perceptron()

In [546]:
ptron.fit(X,y)

In [383]:
pred = ptron.predict(test_final_df)

In [384]:
#Si se quiere hacer un submit solo con perceptron, pasar directo al Armado de Submit

# Bagging con Perceptron

In [218]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=ptron,n_estimators=50)

In [219]:
bc.fit(X,y)

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

BaggingClassifier(base_estimator=Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [220]:
pred = bc.predict_proba(test_final_df)

In [221]:
pred = pred[:,1]

# Perceptron multinivel

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier()

In [ ]:
mlp.fit(X,y) 

In [ ]:
pred = mlp.predict(test_final_df)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=50)

In [ ]:
knn.fit(X,y)

In [ ]:
knn.score(X,y)

In [ ]:
knn.predict(test_final_df)

# Bagging KNN

In [ ]:
from sklearn.ensemble import BaggingClassifier
bcknn = BaggingClassifier(base_estimator=knn,n_estimators=20)

In [ ]:
bcknn.fit(X,y)

In [ ]:
bcknn.score(X,y)

In [ ]:
pred=bcknn.predict_proba(test_final_df)

# Naive_Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
matrix_df.head()

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
X.columns

In [ ]:
#from sklearn.preprocessing import StandardScaler

#features = ['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral','nombre_area','sexo','nombre','estado','edad']
#X = X.loc[:, features].values
#X = StandardScaler().fit_transform(X)
#X

In [ ]:
#X = pd.DataFrame(X)
#X.columns = ['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral','nombre_area','sexo','nombre','estado','edad']

#X.head()

In [ ]:
y.value_counts()

In [ ]:
nb.fit(X, y)

In [ ]:
nb.score(X,y)

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
pred = nb.predict(test_final_df)

# Boosting

In [191]:
from sklearn.ensemble import AdaBoostClassifier



#.drop(['interes_en_titulo_de_aviso','relacion_educacion_con_aviso'],axis=1)

adaboost = AdaBoostClassifier(n_estimators=100)
 

In [192]:
from sklearn.model_selection import cross_val_score

In [193]:
scores = cross_val_score(adaboost,X,y)
scores.mean()  

0.9044846732046948

In [194]:
adaboost.fit(X,y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [195]:
test_final_df.head()

,postulaciones_a_ese_area,postulaciones_a_esa_empresa,interes_en_titulo_de_aviso
0,0,0,0
1,5,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [197]:
pred = adaboost.predict_proba(test_final_df)

In [198]:
pred = pred[:,1]

In [196]:
list(zip(X, adaboost.feature_importances_))

[('postulaciones_a_ese_area_en_esa_zona', 0.32),
 ('postulaciones_a_esa_empresa', 0.1),
 ('interes_en_titulo_de_aviso', 0.58)]

# Gradient Boosting

In [518]:
from sklearn.ensemble import GradientBoostingClassifier

In [519]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=0)

In [521]:
gb.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [526]:
pred = gb.predict_proba(test_final_df)

In [527]:
pred = pred[:,1]

In [540]:
scores = cross_val_score(gb, X.drop(['postulaciones_a_esa_empresa','relacion_educacion_con_aviso'],axis=1),y)
scores

array([0.88849223, 0.88884889, 0.88647886])

# ARMADO DE SUBMIT

In [207]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [208]:
test_final_df['sepostulo'] = pd.DataFrame(pred)

In [209]:
submit = test_final_df.drop(['idaviso','idpostulante'],axis=1)

In [210]:
submit.set_index('id',inplace=True)

In [211]:
submit.head()

,sepostulo
id,
0,0.489737
1,0.495989
2,0.489737
3,0.489737
4,0.489737


In [212]:
submit.sepostulo.value_counts()

0.489737    47285
0.496693     2953
0.511041     2226
0.509784     1968
0.508740     1934
0.512377     1763
0.510504     1626
0.495452     1496
0.498226     1318
0.510321     1259
0.512556     1244
0.511220     1148
0.511840      861
0.502409      797
0.510683      791
0.508199      786
0.499413      696
0.509963      665
0.498871      661
0.493348      650
0.509277      620
0.508919      609
0.507553      600
0.503942      521
0.510500      511
0.511958      510
0.512019      502
0.513213      436
0.500457      434
0.510874      433
            ...  
0.519274        1
0.595262        1
0.541190        1
0.510699        1
0.518843        1
0.507741        1
0.501550        1
0.507092        1
0.525122        1
0.517705        1
0.502454        1
0.509938        1
0.482710        1
0.515971        1
0.517878        1
0.564903        1
0.612757        1
0.607113        1
0.473630        1
0.517004        1
0.507982        1
0.512748        1
0.508437        1
0.504781        1
0.518542  

In [213]:
submit.head()

,sepostulo
id,
0,0.489737
1,0.495989
2,0.489737
3,0.489737
4,0.489737


In [214]:
submit.to_csv('Submit_file.csv')